In [23]:
!pip install autopep8
!pip install jupyter_contrib_nbextensions==0.5.1
!pip install bokeh==0.12.15

In [ ]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

from dwave.system.composites import EmbeddingComposite
from collections import OrderedDict

import dwavebinarycsp as dbc

import time

def to_base_ten(sample, module_length): 
    a = b = 0

    # we know that multiplication_circuit() has created these variables
    a_vars = ["a"+str(i) for i in range(int(module_length/2))]
    b_vars = ["b"+str(i) for i in range(int(module_length/2))]

    for lbl in reversed(a_vars):
        a = (a << 1) | sample[lbl]
    for lbl in reversed(b_vars):
        b = (b << 1) | sample[lbl]

    return a, b

def response_to_dict(sampleset,module_length):
    results_dict = OrderedDict()
    for sample, energy in sampleset.data(['sample', 'energy']):
        a, b = to_base_ten(sample, module_length)
        if (a, b) not in results_dict:
            results_dict[(a, b)] = round(energy, 2)
            
    return results_dict

def fact(P,module_length):
    csp = dbc.factories.multiplication_circuit(int(module_length/2))
    bqm = dbc.stitch(csp, min_classical_gap=.1)
    p_vars = ['p'+str(i) for i in range(module_length)]
    # Convert P from decimal to binary
    fixed_variables = dict(zip(reversed(p_vars), ("{:0"+str(module_length)+"b}").format(P)))
    fixed_variables = {var: int(x) for(var, x) in fixed_variables.items()}
    # Fix product variables
    for var, value in fixed_variables.items():
        bqm.fix_variable(var, value)   
    sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')
    sampleset = EmbeddingComposite(sampler).sample(bqm, num_reads=10000)
    results = response_to_dict(sampleset, module_length)
    return list(results.items())[0][0][0],list(results.items())[0][0][1]

# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = "amazon-braket-zolnierczyk" # the name of the bucket
my_prefix = "factoring" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

# session and device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
#print('Device:', device)

and_gate = dbc.factories.and_gate(["x1", "x2", "x3"])
and_csp = dbc.ConstraintSatisfactionProblem('BINARY')
and_csp.add_constraint(and_gate)
# Test the CSP
and_csp.check({"x1": 1, "x2": 1, "x3": 1})


L10 = [493, 551, 437, 391, 323, 667]
L12 = [2183, 1927, 2623, 2491, 2501, 2257, 2419, 1961, 2773, 2173, 2867, 2279, 3233, 1517, 3599, 2021, 3127, 1739, 2537, 1591, 1763]
L14 = [7313, 6767, 10379, 5609, 9523, 8023, 8383, 8989, 9701, 7597, 8549, 4891, 6497, 7957, 7303, 5183, 7739, 5893, 11413, 8611] 
L16 = [35621, 34579, 33017, 26069, 32111, 38009, 20989, 19519, 27641, 33043, 34189, 29353, 36349, 37627, 23213, 56153, 41347]

P10 = 0
P12 = 0
P14 = 0
P16 = 0

T10 = 0
T12 = 0
T14 = 0
T16 = 0

L = [L10,L12,L14, L16]
P = []
T = []
  
module_length = 8
for l in L:
    succes = 0
    nprob = 0
    czas = 0
    module_length = module_length +2
    while succes<3:        
        for n in l:
            start = time.time()            
            a,b = fact(n,module_length)
            stop = time.time()
            czas = czas+stop-start
            #print(a,b,end=',')
            if n % a ==0 or n % b ==0:
                succes = succes +1
                print("SUKCES!")
        nprob = nprob+len(l)        
    T.append(czas/nprob)
    P.append(succes/nprob) 

print(T)
print(P)



SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
